In [18]:
import os
from dotenv import load_dotenv
load_dotenv()
SUMMARIZE_MODEL_OCID = os.getenv('SUMMARIZE_MODEL_OCID') 
GEN_AI_INFERENCE_ENDPOINT = os.getenv('GEN_AI_INFERENCE_ENDPOINT') 
COMPARTMENT_ID = os.getenv('COMPARTMENT_ID') 
GENERATION_MODEL_OCID = os.getenv('GENERATION_MODEL_OCID') 
GEN_AI_ENDPOINT = os.getenv('GEN_AI_ENDPOINT') 
GENERATION_MODEL_OCID_llam = os.getenv('GENERATION_MODEL_OCID_llam') 



import oci
config = oci.config.from_file()




In [19]:
import textwrap

# Function to wrap text at 80 characters per line while keeping sentences intact
def wrap_text_by_sentence(text, target_language_code, width=80):
    if target_language_code == 'ja':
        sentences = text.split("。")  # Split text into sentences based on period (。)
    else:
        sentences = text.split(". ") 
    wrapped_sentences = []
    for sentence in sentences:
        if sentence:
            # Wrap each sentence with a width of 80 characters
            wrapped_sentence = textwrap.fill(sentence + "。", width=width)
            wrapped_sentences.append(wrapped_sentence)
    return "\n".join(wrapped_sentences)


## translate with OCI

In [20]:

# translate methord made with ai_language
def translate_and_wrap_text(text_from, compartment_id, source_language_code, target_language_code):
    # Initialize service client with default config file
    ai_language_client = oci.ai_language.AIServiceLanguageClient(config)
    
    # Send the request to service, some parameters are not required, see API doc for more info
    batch_language_translation_response = ai_language_client.batch_language_translation(
        batch_language_translation_details=oci.ai_language.models.BatchLanguageTranslationDetails(
            documents=[
                oci.ai_language.models.TextDocument(
                    key="1",
                    text=text_from,
                    language_code=source_language_code)],
            compartment_id=compartment_id,
            target_language_code=target_language_code),
        opc_request_id="VUPH6Z9C2LD99QDZBJODaaaaaaaamlgyvhoa4qvxzsoguxpdo62juyzpnaxscwq3n5kaxptrha5ihy4a")
    
    # Get the data from response
    translated_text = batch_language_translation_response.data.documents[0].translated_text
    
    # Wrap the translated text by sentence
    wrapped_text = wrap_text_by_sentence(translated_text,target_language_code)
    
    return wrapped_text



In [21]:

# Example usage
text_from = "Proper waste management is essential to the health of all living systems. Our grantees include Wecyclers, a social enterprise in Lagos, Nigeria, that incentives waste recycling in densely populated urban neighbourhoods, and the WEEE Centre in Nairobi, Kenya, which promotes public awareness of the environmental and health hazards of e-waste and educates the public about reuse, recycling, and safe disposal. Oracle is also a founding sponsor of California Coastal Cleanup Day, which has removed hundreds of thousands of pounds of trash from California waterways each year since 1995"
source_language_code = "en"
target_language_code = "ja"

translated_and_wrapped_text = translate_and_wrap_text(text_from, COMPARTMENT_ID, source_language_code, target_language_code)
print(translated_and_wrapped_text)

適切な廃棄物管理は、すべての生活システムの健全性に不可欠です。
私たちの助成金には、ナイジェリアのラゴスの社会企業であるウェサイクラーズ、人口密集した都市部での廃棄物リサイクルを奨励するウェサイクラーズ、ケニアのナイロビのW
EEEセンター、電子廃棄物の環境および健康上の危険に対する公衆の認識を促進し、再利用、リサイクル、および安全な処分について市民に教育する。
OracleはCalifornia Coastal Cleanup
Dayの創設スポンサーでもあり、1995年以来、毎年何十万ポンドものゴミをCalifornia Waterwaysから削除しています。


In [22]:
# Example usage
text_from = "適切な廃棄物管理は、すべての生活システムの健全性に不可欠です"
source_language_code = "ja"
target_language_code = "en"

translated_and_wrapped_text = translate_and_wrap_text(text_from, COMPARTMENT_ID, source_language_code, target_language_code)
print(translated_and_wrapped_text)

Proper waste management is essential to the health of all living systems。


##  generate with meta.llama-2-70b-chat

In [23]:
def generate_job_description(llam_model_ocid, endpoint, compartment_id, prompt):
    generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
        config=config,
        service_endpoint=endpoint,
        retry_strategy=oci.retry.NoneRetryStrategy(),
        timeout=(10, 240)
    )

    generate_text_detail = oci.generative_ai_inference.models.GenerateTextDetails()
    llam_inference_request = oci.generative_ai_inference.models.LlamaLlmInferenceRequest()
    llam_inference_request.prompt = prompt

    llam_inference_request.max_tokens = 600
    llam_inference_request.temperature = 1
    llam_inference_request.frequency_penalty = 0
    llam_inference_request.top_p = 0.75

    generate_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=llam_model_ocid)
    generate_text_detail.inference_request = llam_inference_request
    generate_text_detail.compartment_id = compartment_id

    generate_text_response = generative_ai_inference_client.generate_text(generate_text_detail)

    return generate_text_response.data




In [24]:
# Example usage

prompt = """
Generate a job description for a data visualization expert with the following three qualifications only:
1) At least 5 years of data visualization expert
2) A great eye for details
3) Ability to create original visualizations
"""

generated_text = generate_job_description(GENERATION_MODEL_OCID_llam, GEN_AI_INFERENCE_ENDPOINT, COMPARTMENT_ID,prompt)


In [25]:
print("**************************Generate Texts Result**************************")
generated_result = generated_text.inference_response.choices[0].text
print(generated_result)

**************************Generate Texts Result**************************

Job Title: Data Visualization Expert

Job Summary:
We are seeking a highly skilled and experienced Data Visualization Expert to join our team. The successful candidate will have at least 5 years of experience in data visualization, a great eye for details, and the ability to create original visualizations. The Data Visualization Expert will be responsible for creating and maintaining high-quality data visualizations that effectively communicate complex data insights to both technical and non-technical audiences.

Responsibilities:

* Design and develop data visualizations that effectively communicate complex data insights to both technical and non-technical audiences.
* Work closely with data analysts and data scientists to understand data sets and identify key insights.
* Create original visualizations that are both aesthetically pleasing and informative.
* Ensure that all visualizations are accurate, up-to-dat

##  generate with cohere

In [26]:
class GenerativeAiClient:
    def __init__(self, config, service_endpoint):
        self.config = config
        self.service_endpoint = service_endpoint

    def generate_job_description(self, prompt, compartment_id, serving_mode, model_id, max_tokens, num_generations):
        gen_ai_inference_client = GenerativeAiInferenceClient(
            config=self.config,
            service_endpoint=self.service_endpoint
        )

        generate_text_response = gen_ai_inference_client.generate_text(
            generate_text_details=GenerateTextDetails(
                compartment_id=compartment_id,
                serving_mode=serving_mode,
                inference_request=CohereLlmInferenceRequest(
                    prompt=prompt,
                    is_stream=False,
                    max_tokens=max_tokens,
                    num_generations=num_generations
                )
            )
        )

        return generate_text_response.data




In [27]:
# Example usage
gen_ai_client = GenerativeAiClient(config=config, service_endpoint=GEN_AI_INFERENCE_ENDPOINT)

prompt = """
Generate a job description for a data visualization expert with the following three qualifications only:
1) At least 5 years of data visualization expert
2) A great eye for details
3) Ability to create original visualizations
"""

serving_mode = OnDemandServingMode(model_id=GENERATION_MODEL_OCID)
max_tokens = 906
num_generations = 1

generated_text = gen_ai_client.generate_job_description(prompt, COMPARTMENT_ID, serving_mode, GENERATION_MODEL_OCID, max_tokens, num_generations)
print(generated_text)

{
  "inference_response": {
    "generated_texts": [
      {
        "finish_reason": null,
        "id": "0ff0fbe3-1123-4a4c-acb3-45ba45b993c3",
        "likelihood": null,
        "text": " We're looking for a talented Data Visualization Expert to join our team! The ideal candidate will have at least 5 years of experience in creating compelling and effective data visualizations, as well as an excellent eye for detail and the ability to create original visualizations that effectively communicate complex data insights.\n\nAs a Data Visualization Expert, you will be responsible for designing and developing visual representations of data and information to help stakeholders understand and interpret important trends and insights. You will work closely with our team of data analysts and scientists to understand the data and the story it needs to tell.\n\nYour responsibilities will include:\n\nDeveloping creative and innovative data visualizations that effectively communicate complex data i

In [ ]:
import textwrap
import re

def wrap_text_by_sentence(text, target_language_code, width=60):
    # Choose the correct delimiter based on the target language
    if target_language_code == 'ja':
        splitStr = "。|！|？|  "  # Split text into sentences based on period (。), exclamation mark (！), question mark (？), and double space ( )
    else:
        splitStr = ".|!|?|,| "  # Split text into sentences based on period (.), exclamation mark (!), question mark (?), comma (,), and space ( )

    # Use regular expression to split the text
    sentences = re.split(f'({splitStr})', text)

    # Initialize variables to store wrapped text
    wrapped_text = ''
    line_length = 0

    # Iterate through the split sentences
    for token in sentences:
        # If the sentence is not empty and adding its length to the current line length does not exceed the specified width,
        # add the sentence to the current line
        if token and line_length + len(token) <= width:
            wrapped_text += token
            line_length += len(token)
        # If the sentence is not empty, but adding its length to the current line length exceeds the specified width,
        # add the sentence to a new line
        elif token:
            wrapped_text += '\n' + token
            line_length = len(token)

    return wrapped_text